Author: Hasim Engin <br>
email:hengin@ciesin.columbia.edu<br>
Project: GRID3<br>
Organization:CIESIN, Columbia University<br>

# Create Catchments for a PoI 

This notebook creates a point of interest geographic coverage based on both distance (by time) and capacity.
## Input layers:

* **Friction layers**: Friction layers (walk and walk+motorized) in minutes.They can be created by running "create_friction_surface" script.
* **PoI layer**: A point of interest layer. Such as health facilities, schools, water sources .... POI layer must have unique id and capacity for each record
* **Population raster layer**: It should be in the same resolution with friction layers (100m). Building foorprint constrained population layers that WorldPop produces are suggested. They can be downloaded here:https://www.worldpop.org/project/categories?id=3


## Output layers: 

Output layers will be saved into a directory that is created specified workspace directory.A gdb will be created in the directory to save temporal layers. It can be deleted if output raster layer is ok.
 <br>
* **output.gbd** : The scripts run trought each record in PoI layer and calculates service area for each record based on specified time in minutes and capacity. In order to avoid double count of popaultion in each serves area, covered population exluded after each itaration. 
 <br>
* **Final Layers** : <br>
   ..._geographic_coverage.tif: This is the final raster layer that shows options of PoI that people can go in specified distance (by time) and capacity of PoI .<br>
   ..._geographic_coverage_summary.csv: This is a summary table that calcualtes population total by the number of options that they can get a service from input PoI.
   
    
    


## import Libraries

In [ ]:
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

In [ ]:
# required libraries can be installed with following code
#uncommaent the line below and write a -library name- after !):
#! -library name- install

## Functions

In [ ]:


###==================================== functions =============================================###

def delete_Dataset(datasetList):
    for dst in datasetList:
        #print(dst)
        if arcpy.Exists(dst):
            arcpy.Delete_management(dst)
def facility_geographic_coverage(PoI_layer,PoI_type,  PoI_unique_id, friction_surface, pop_raster, time_limit, capacity_var,output_loc):
    '''
    calculate a facility accessibility based on distance in time and capacity of facility.
    -------------------------------------------------------------------------------------------
    inputs:
    facility_loc: point layer of facility location.
    facility_unique_id: unique id for each facility. "FID" can be used for a shp file, or
        "OBJECTID" for a feature layer.
    friction_surface: friction raster layer. It can be walk or combiantion walk and walk+motorized
    pop_raster: population raster layer. it should has  the same resolution with accessibility raster.
    time_limit: upper limit of time in minutes.
    capacity_var: column in 
    output:
    catchment of facility as a raster layer facility_loc that indicates capacity of facility
    '''
    ##=======================================================================#

    # set enviroment for raster processing
    arcpy.env.snapRaster  =friction_surface
    arcpy.env.extent      =friction_surface
    arcpy.env.cellSize = friction_surface
    arcpy.env.cellAlignment = friction_surface
    arcpy.env.outputCoordinateSystem = arcpy.Describe(friction_surface).spatialReference

    #------------------------------------------------------------------------#

    # preprocessing:
        # created list of facilities to process a facility a time. 

    facility_ids=pd.DataFrame.spatial.from_featureclass(PoI_layer)[ PoI_unique_id].tolist()
    facility_count=len(facility_ids)
    print (f"{facility_count} will be processed. It may take some time")
    
    #------------------------------------------------------------------------#
        # copy population raster to workspace
    if not arcpy.Exists("pop_layer"):
        print ("pop layer ready")
        out_coor_system = arcpy.Describe (friction_surface).spatialReference
        arcpy.management.ProjectRaster(pop_raster, "pop_layer",out_coor_system)   

    #------------------------------------------------------------------------#
    for _id in facility_ids:
        #print (_id)
        if not arcpy.Exists("cathchment_"+str(_id)):
           
            try:
                facility_point=arcpy.Select_analysis(PoI_layer,"facility_"+str(_id),"{}= {}".format(PoI_unique_id,_id))
                capacity=int(pd.DataFrame.spatial.from_featureclass(facility_point)[capacity_var].min())
                CostDistance(facility_point,friction_surface,time_limit, "outBkLinkRaster").save("cost_distance_"+str(_id))
                
               
                i=60
                get_pop=999999
                while get_pop>= capacity:
                    
                    arcpy.gp.ExtractByAttributes_sa("cost_distance_"+str(_id), """"Value" <={} """.format(str(i)),"cost_distance")
                    Times("cost_distance", 0).save("cost_mask")
                    Int("cost_mask").save("catchment")
                    ZonalStatistics("catchment", "value", "pop_layer", "SUM","DATA").save("cost_pop")
                    get_pop=int(float(arcpy.GetRasterProperties_management ("cost_pop", "MINIMUM").getOutput (0)))
                    print (" Health facility == {}, distance in min == {}, popualtion =={}, capacity=={}".format(_id,time_limit,get_pop,capacity))
                    #delete_Dataset(["cost_distance_"+str(i),"cost_mask_"+str(i), "catchment_"+str(i)])
                    i=i-3

                #print (" Health facility == {}, distance in min == {}, popualtion =={}, capacity=={}".format(_id,time_limit,get_pop,capacity))
                Plus("catchment",1).save("catchment_"+str(_id))
                Reclassify("catchment", "Value", "0 0;NODATA 1", "DATA").save("catchment_temp")
                Times("catchment_temp", "pop_layer").save("pop_layer_temp")
                arcpy.Delete_management("pop_layer")
                arcpy.Rename_management("pop_layer_temp", "pop_layer")
                #delete_Dataset(["pop_layer_temp","cost_pop_"+str(_id),"cost_pop_mask_"+str(_id),"catchment_temp_"+str(_id)])


            except arcpy.ExecuteError:
                #print(arcpy.GetMessages(2)) 
          
                delete_Dataset([ facility_point, "outBkLinkRaster","cost_distance_"+str(_id)])
            
    #-------------------------------------------------------------------------#
    # merge all individual catchments
    print ("Merging all catcments...")
    merge_catchments=[rs for rs in  arcpy.ListRasters() if rs.startswith("catchment_")]  
    arcpy.sa.CellStatistics(merge_catchments, "SUM", "DATA").save("geographic_coverage_temp")
    Times("pop_layer", 0).save("pop_layer_mask")
    arcpy.sa.CellStatistics(["geographic_coverage_temp","pop_layer_mask"], "SUM", "DATA").save("geographic_coverage_temp2")
    arcpy.sa.ExtractByMask("geographic_coverage_temp2", "pop_layer_mask").save(output_loc+"\\"+PoI_type+"_geographic_coverage.tif")
    #-------------------------------------------------------------------------#
    # get population summary
    print ("Calculating overall population summary...")
    out_coor_system = arcpy.Describe (friction_surface).spatialReference
    arcpy.management.ProjectRaster(pop_raster, "pop_layer2",out_coor_system)
    Int(output_loc+"\\"+PoI_type+"_geographic_coverage.tif").save("catchments_int")
    arcpy.sa.ZonalStatisticsAsTable("catchments_int", "Value","pop_layer2", "pop_summary", "DATA", "SUM")
    arcpy.conversion.TableToTable("pop_summary", output_loc, PoI_type+"_geographic_coverage_summary.csv")
    
    
    #-------------------------------------------------------------------------#
   
    
    print (">>> There are {facility_count} facility , and {len(merge_catchments)} catchments are combined")





## User Inputs

In [ ]:

###=================== Initialize Variables ===================###


poi_type=  ""           # type of point of interest such as "health_facility"      !! no space between names !!
PoI_unique_id=  ""      # unique id of each PoI, OBJECTID or FID can be used if there is no any
capacity_var=""         # capacity of each PoI
time_limit=""            # distance interval in minutes
travel_type= ""         # options: walk or mix( walk+motorized)                      

###================================ input layers ==============================================###

# path to PoI layer such as health facilities or schools
path_to_poi=r""

# friction surface for walking. It is created by "create_friction_surface" script
path_to_friction_walk=r""

# friction surface for motorized.It is created by "create_friction_surface" script
path_to_friction_mix=r""

#path to population raster
path_to_pop_raster=r"" # WorldPop building footprint contraint population layer

# path to the directory that you want to save outputs
output_path=r""

## Create Output Workspace

In [ ]:
 #create output directory
if not  os.path.exists (os.path.join(output_path,"output")):
    os.mkdir(os.path.join(output_path,"output"))
output_loc=os.path.join(output_path,"output")

 #create output gdb
if  not arcpy.Exists(os.path.join(output_loc,"output.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"output.gdb")
output_gdb=os.path.join(output_loc,"output.gdb")

# workspace
arcpy.env.workspace=output_gdb

## Process

In [ ]:


###==================== select friction layer based on travel type============================###

if travel_type=="walk":
    friction_layer= path_to_friction_walk
if travel_type=="mix":
    friction_layer= path_to_friction_mix
            
###======================== prosessin ==============================###

print ("========================= PROCESSING START ====================================")

facility_geographic_coverage(path_to_poi, PoI_unique_id, friction_layer,path_to_pop_raster, time_limit,
                             capacity_var, output_loc)

#delete output gdb 
#arcpy.Delete_management(output_gdb)

print ("SCRIPT is DONE!!")
    



